# 集成学习

### 1. 为什么要集成？集成有什么好处？

我们的问题可以描述为：

**数据**：训练集$\mathcal{D}=\{(X_1, y_1), (X_2, y_2),\cdots,(X_n, y_n)\}$

**模型**：$y_i=f(X_i)+\epsilon_i, i\in \{1,2,\cdots,n\}$

$\qquad X_1, X_2,\cdots, X_n \overset{\text{i.i.d}}{\sim} F$

$\qquad\epsilon_1, \epsilon_2, \cdots, \epsilon_n \overset{\text{i.i.d}}{\sim} (0,\sigma^2)$

**目标**：学习一个函数$\hat{f}$，对新的来自$F$的数据$\tilde{X}$进行预测，使得$(y-\hat{f}(\tilde{X}))^2$尽可能小

使用单个模型时，我们有**偏差-方差分解定理**。

<div class="alert alert-info"><h4>定理1 (偏差-方差分解定理)</h4><p>
    在给定$\tilde{X}$的情况下，对于平均平方损失$L(\hat{f})=E_{\mathcal{D}}(y-\hat{f}_{\mathcal{D}}(\tilde{X}))$，我们有分解<br><br>
    $$\begin{align*}
    L(\hat{f}) & = E_{\mathcal{D}}(y-\hat{f}_{\mathcal{D}})^2 \\
    & = E_{\mathcal{D}}(f+\epsilon-\hat{f}_{\mathcal{D}}+E_{\mathcal{D}}[\hat{f}_\mathcal{D}]-E_{\mathcal{D}}[\hat{f}_{\mathcal{D}}])^2 \\
    & = E_{\mathcal{D}}[(f-E_{\mathcal{D}}[\hat{f}_{\mathcal{D}}])+(E_{\mathcal{D}}[\hat{f}_\mathcal{D}]-\hat{f}_{\mathcal{D}})+\epsilon]^2 \\
    & = E_{\mathcal{D}}[(f-E_{\mathcal{D}}[\hat{f}_{\mathcal{D}}])^2]+
    E_{\mathcal{D}}[(E_{\mathcal{D}}[\hat{f}_\mathcal{D}]-\hat{f}_\mathcal{D})^2]+
    E_{\mathcal{D}}[\epsilon^2] \\
    & = [f-E_{\mathcal{D}}[\hat{f}_{\mathcal{D}}]]^2+E_{\mathcal{D}}[(E_{\mathcal{D}}[\hat{f}_{\mathcal{D}}]-\hat{f}_{\mathcal{D}})^2]+\sigma^2
    \end{align*}$$
</p></div>

简单地说，一个模型的平方损失来源于三个部分：

(1).数据真实值与模型的平均预测值的平方误差，即模型偏差的平方$[f-E_{\mathcal{D}}[\hat{f}_{\mathcal{D}}]]^2$

(2).模型预测值的方差$E_{\mathcal{D}}[(E_{\mathcal{D}}[\hat{f}_{\mathcal{D}}]-\hat{f}_{\mathcal{D}})^2]=\text{Var}(\hat{f}_{\mathcal{D}})$

(3).数据本身存在的噪声的方差$\sigma^2$

(3)是不可能通过优化模型解决的，因此主要思路应该是尽量减小模型的偏差和方差

由于单个模型很难同时控制偏差和方差，因此一个想法是，**利用多个偏差较小的模型，通过平均或其他方式集成降低方差**。

<div class="alert alert-block alert-success"><h4>练习1</h4><p>
    说明偏差-方差分解定理中第四个等号为何成立。
</p></div>

要证明第四个等号成立，只需要证明开平方的交叉项为0即可。

需要注意到，$\hat{f}_{\mathcal{D}}$只与$\mathcal{D}$有关，也就是只与$X_1,\cdots,X_n$和$\epsilon_1,\cdots,\epsilon_n$有关，$f$在给定$\tilde{X}$的情况下就是常数，而$\epsilon$是预测误差，与$\epsilon_1,\cdots,\epsilon_n$无关。

$$\begin{align*}
E_{\mathcal{D}}[(f-E_{\mathcal{D}}[\hat{f}_{\mathcal{D}}])(E_{\mathcal{D}}[\hat{f}_\mathcal{D}]-\hat{f}_\mathcal{D})] & = E_{\mathcal{D}}[f(E_{\mathcal{D}}[\hat{f}_\mathcal{D}]-\hat{f}_\mathcal{D})] \\
& = E_{\mathcal{D}}f*E_{\mathcal{D}}\hat{f}_{\mathcal{D}}-E_{\mathcal{D}}[f]*E_{\mathcal{D}}[\hat{f}_\mathcal{D}]=0\\
\end{align*}$$

$$E_{\mathcal{D}}[(f-E_{\mathcal{D}}[\hat{f}_{\mathcal{D}}])\epsilon]=(f-E_{\mathcal{D}}[\hat{f}_{\mathcal{D}}])E_{\mathcal{D}}[\epsilon]=0$$

$$E_{\mathcal{D}}[(E_{\mathcal{D}}[\hat{f}_\mathcal{D}]-\hat{f}_\mathcal{D})\epsilon]=-E_{\mathcal{D}}[\hat{f}_{\mathcal{D}}\epsilon]=0$$

<div class="alert alert-block alert-success"><h4>练习2</h4><p>
    有人说，Bias-Variance Tradeoff就是指“一个模型要么具有大的偏差，要么具有大的方差”，你认为这种说法对吗？<br><br>你能否对“偏差-方差权衡”现象做出更准确的表述？
</p></div>

这样的说法显然是不对的，一个糟糕的模型完全可以既有大的偏差又有大的方差，这样的模型是完全没有用的。

更准确的说法是，如果我们希望模型的偏差得到足够的改善，那么需要容许模型的方差有可能的增大，反之亦然。

### 2. Bagging和Boosting集成方法

**Bagging的思路是：利用Bootstrap抽样，得到$M$组数据集，一组数据集训练一个基学习器，再将$M$个不同的基学习器结果汇总**

如回归任务中，假如每个基学习器方差均为$\sigma^2$，基学习器两两相关系数为$\rho$，汇总输出方式为取平均值，则

$$\text{Var}(\hat{y})=\rho\sigma^2+(1-\rho)\frac{\sigma^2}{M}$$

只要$\rho\neq 1$，就可以减小方差。**基学习器之间相关度越小，bagging效果越好**。

在Bootstrap抽样中，假设总体为$n$个样本，有放回抽取$n$个样本，则单个样本被抽取的概率是

$$1-(1-\frac{1}{n})^n \to 1-e^{-1}$$

**Boosting的思路是：第一个学习器进行学习，结果由第二个学习器进行校正以减少损失，校正后的结果继续由第三个学习器接着校正，以此类推**

由于总体模型经过了多次的校正，因此可以使得偏差降低。多个学习器的结果实现加权也有助于降低方差。

<div class="alert alert-block alert-success"><h4>练习3</h4><p>
    假设总体有100个样本，每轮利用bootstrap抽样从总体中得到10个样本（即可能重复），请求出所有样本都被至少抽出过一次的期望轮数。<br><br>（通过本文介绍的方法，我们还能得到轮数方差的bound）
</p></div>

问题等价于100个样本有放回抽取，所有样本都被至少抽出过一次的期望抽样次数，得到的结果除以10即可。

记随机变量$X_i$为抽取到第$i-1$个新样本到抽取到第$i$个新样本的抽取次数，则总抽取次数$S=\sum_{i=1}^{100}X_i$

$X_i\sim Geo(\frac{101-i}{100})$，因此$EX_i=\frac{100}{101-i}$，

$$ES=\sum_{i=1}^{100}EX_i=\sum_{i=1}^{100}\frac{100}{101-i}=100\sum_{i=1}^{100}\frac{1}{i}\approx 100\ln100\approx 460.5$$

因此所有样本被至少抽出过一次的期望轮数约为46轮。

### 3. Stacking和Blending集成方法

**Stacking的思路是：每个基学习器对训练集进行K折交叉验证，同时分别对测试集预测K次取平均，各个基学习器对训练集的预测组成特征，对测试集的结果组成标签，使用最终模型进行训练，具体操作可以见下图。**

![avatar](https://pic2.zhimg.com/v2-c5f0603eea712f87fc9b1c3f980d354d_r.jpg)

显而易见的是，Stacking有着很好的性质，多次的交叉验证确保了模型的可靠，是数据竞赛的大杀器。

但是，Stacking的缺点也很明显，训练和预测次数过多，计算复杂度非常高。Blending方法缓解了这个问题。

**Blending的思路是：不再使用K折交叉验证，而是直接按比例划分训练集和验证集，最终训练中只组合验证集的结果作为训练样本。**

好处是很大程度上减少了训练和预测的次数，问题是没有充分使用训练数据。

因此在实际中，有时会在部分数据上Stacking，部分数据Blending，组合使用。

<div class="alert alert-block alert-success"><h4>练习4</h4><p>
    对于stacking和blending集成而言，若$m$个基模型使用$k$折交叉验证，此时分别需要进行几次训练和几次预测？
</p></div>

对于Stacking集成，每个基模型训练$k$次，共$m$个模型，加上最后训练的一次，总共$(mk+1)$次训练<br><br>
每个基模型既要预测验证集又要预测测试集，加上最后预测的一次，总共$(2mk+1)$次预测<br><br>
对于Blending集成，每个基模型训练1次，加上最后的一次，总共$(m+1)$次训练，$(2m+1)$次预测

### 知识回顾

<div class="alert alert-block alert-success"><h4>知识回顾1</h4><p>
    什么是偏差和方差分解？偏差是谁的偏差？此处的方差又是指什么？
</p></div>

偏差方差分解，指的是模型的平方损失可以分解为三个部分，分别是模型偏差的平方，模型的方差以及数据的噪声。<br><br>
偏差指的是模型的偏差，即数据真实值与模型的平均预测值之差的平方，方差指的是模型输出的方差

<div class="alert alert-block alert-success"><h4>知识回顾2</h4><p>
    相较于使用单个模型，bagging和boosting方法有何优势？
</p></div>

Bagging和Boosting方法通过集成模型的方式，能够在保证偏差的同时降低方差

<div class="alert alert-block alert-success"><h4>知识回顾3</h4><p>
    请叙述stacking的集成流程，并指出blending方法和它的区别。
</p></div>

Stacking的集成：<br><br>
(1).每个模型利用K折交叉验证，用训练集训练模型并对验证集和测试集都进行预测，取遍K份验证数据<br><br>
(2).每个模型的K份验证数据拼接为该模型在总模型上的训练集特征，测试集的K份预测数据取平均作为该模型在总模型上的测试集特征<br><br>
(3).使用最终模型，以训练集的样本标签为目标进行训练，得到最终的stacking模型<br><br>
Blending的区别在于，不再使用K折交叉验证，而是直接按比例划分训练集和验证集，最终训练中只组合验证集的结果作为训练样本。